# IN THIS PARTICULAR NOTEBOOK I AM TRYING TO CREATE AN AI DETECTOR WHICH TAKES TEXT AS INPUT AND PREDICTS ITS HUMAN WRITTEN OR AI WRITTEN
**Particularly its a difficult task to accomplish with Only Classical ML but i was learning forward before applying it to big level i thought to just give a try using my ML skills as I know to create a detector is possible may be not that high accuracy but that much atleast it can perform on the data on which it is trained.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORTING DATASET
**For this i am using hugging face dataset for training the model having rows around 850. This data includes questions and answers in which one answer is human written or human answer and one is Chat GPT answers**

In [2]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("Hello-SimpleAI/HC3", name="wiki_csai", split='train')
print(dataset[0])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-

README.md:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

HC3.py:   0%|          | 0.00/9.47k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/842 [00:00<?, ? examples/s]

{'id': '0', 'question': 'Please explain what is "Animal cognition"', 'human_answers': ['Animal cognition encompasses the mental capacities of non-human animals including insect cognition. The study of animal conditioning and learning used in this field was developed from comparative psychology. It has also been strongly influenced by research in ethology, behavioral ecology, and evolutionary psychology;  the alternative name cognitive ethology is sometimes used. Many behaviors associated with the term animal intelligence are also subsumed within animal cognition.\nResearchers have examined animal cognition in mammals (especially primates, cetaceans, elephants, dogs, cats, pigs, horses, cattle, raccoons and rodents), birds (including parrots, fowl, corvids and pigeons), reptiles (lizards, snakes, and turtles), fish and invertebrates (including cephalopods, spiders and insects).'], 'chatgpt_answers': ['Animal cognition refers to the mental capacities of non-human animals. It includes the

# MODEL TRAINING 
**Preprocessing and combining the text data.**

**Converting the text to numerical features using TF-IDF VECTORIZER.**

**Training a Random Forest model on this data.**

**Evaluating the model's ability to correctly identify the source of new text.**

I use a Random Forest Classifier, which is an ensemble learning method. It combines the predictions of multiple decision trees to improve accuracy and reduce overfitting.

🔹 Some of the advantages due to which i used it are :

**Handles high-dimensional data well (like text after TF-IDF).**

**Robust to noise and overfitting.**

**Easy to interpret feature importance.**

In [3]:
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import classification_report


data = []
for example in dataset:
    for ans in example['chatgpt_answers']:
        data.append({'text': ans, 'label': 0})  # 0 = AI
    for ans in example['human_answers']:
        data.append({'text': ans, 'label': 1})  # 1 = Human

df = pd.DataFrame(data)

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['text'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

def predict_text_origin(text):
    vec = vectorizer.transform([text])
    pred = model.predict(vec)[0]
    return "Human" if pred == 1 else "AI"

print(predict_text_origin("Human intelligence is the mental ability to think, learn, and understand concepts. It is what enables us to process, analyze, and use information, as well as to reason, plan, solve problems, and make decisions. Intelligence is a complex and multifaceted concept that is difficult to define and measure, but it is generally considered to include a range of cognitive abilities such as verbal and mathematical aptitude, spatial awareness, memory, and problem-solving skills. Intelligence is an important aspect of human behavior that is thought to be influenced by a combination of genetic and environmental factors."))


              precision    recall  f1-score   support

           0       0.86      0.89      0.88       171
           1       0.88      0.86      0.87       166

    accuracy                           0.87       337
   macro avg       0.87      0.87      0.87       337
weighted avg       0.87      0.87      0.87       337

AI


 # EVALUATION
**After training the model on 80% of the data and testing on the remaining 20%, we achieve promising results with high precision and recall for both classes:**

              precision    recall  f1-score   support

           0       0.86      0.89      0.88       ...
           1       0.88      0.86      0.87       ...

    accuracy                           0.87       
This indicates the model can effectively distinguish between AI and human-generated content.

In [4]:
print(predict_text_origin("Human intelligence is the intellectual capability of humans, which is marked by complex cognitive feats and high levels of motivation and self-awareness. High intelligence is associated with better outcomes in life.\nThrough intelligence, humans possess the cognitive abilities to learn, form concepts, understand, apply logic and reason, including the capacities to recognize patterns, plan, innovate, solve problems, make decisions, retain information, and use language to communicate. There are conflicting ideas about how intelligence is measured, ranging from the idea that intelligence is fixed upon birth, or that it is malleable and can change depending on an individual's mindset and efforts. Several subcategories of intelligence, such as emotional intelligence or social intelligence, are heavily debated as to whether they are traditional forms of intelligence. They are generally thought to be distinct processes that occur, though there is speculation that they tie into traditional intelligence more than previously suspected."))

Human


# FINAL PREDICTIONS 
**IT IS PERFORMING GOOD ON THE TRAIN DATA EASILY DIFFRENTIATING BETWEEN AI AND HUMAN ON THE DATA IT WAS TRAINED.**

In [5]:
print(dataset[1])

{'id': '1', 'question': 'Please explain what is "Human intelligence"', 'human_answers': ["Human intelligence is the intellectual capability of humans, which is marked by complex cognitive feats and high levels of motivation and self-awareness. High intelligence is associated with better outcomes in life.\nThrough intelligence, humans possess the cognitive abilities to learn, form concepts, understand, apply logic and reason, including the capacities to recognize patterns, plan, innovate, solve problems, make decisions, retain information, and use language to communicate. There are conflicting ideas about how intelligence is measured, ranging from the idea that intelligence is fixed upon birth, or that it is malleable and can change depending on an individual's mindset and efforts. Several subcategories of intelligence, such as emotional intelligence or social intelligence, are heavily debated as to whether they are traditional forms of intelligence. They are generally thought to be dis